# Build Predictor (XGBoost) Model

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/inference|structured|realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

---

We demonstrate building a ML inference application to predict the rings of Abalone.

After the model is hosted for inference, the payload will be sent as a raw (untransformed) csv string to a real-time endpoint.
The raw payload is first received by the preprocessor container. The raw payload is then transformed (feature-engineering) by the preprocessor, and the transformed record (float values) are returned as a csv string by the preprocessor container.

The transformed record is then passed to the predictor container (XGBoost model). The predictor then converts the transformed record into [`XGBoost DMatrix`](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.DMatrix) format, loads the model, calls `booster.predict(input_data)` and returns the predictions (Rings) in a JSON format.

![Abalone Predictor](../images/byoc-predictor.png)

We use [nginx](https://nginx.org/) as the reverse proxy, [gunicorn](https://gunicorn.org/#deployment) as the web server gateway interface and the inference code as python ["Flask"](https://flask.palletsprojects.com/en/2.3.x/tutorial/factory/) app.

## Dataset and model

For this example, we use a pre-trained [XGBoost](https://xgboost.readthedocs.io) model on [UCI Abalone dataset](https://archive.ics.uci.edu/ml/datasets/abalone).

The pre-trained model accepts input in `text/csv` format and returns prediction results in `application/json`

## Prerequisites

Upgrade the below packages to the latest version.

In [ ]:
!pip install -U awscli boto3 sagemaker watermark scikit-learn tqdm --quiet

%load_ext watermark
%watermark -p awscli,boto3,sagemaker,scikit-learn,tqdm

### Inference script for a predictor (XGBoost) model

- In this example, we download a pre-trained XGBoost model on the UCI abalone dataset from s3.
- The inference code is implemented in [`code/inference.py`](./code/inference.py). The [Flask](https://flask.palletsprojects.com/) app implementation is as follows:
  - Implement routes for `/ping` and `/invocations`
  - Implement functions to handle preprocessing, model loading and prediction
  - Predictions will be returned from `/invocations` function

In [ ]:
import boto3
import sagemaker
import os
from sagemaker import get_execution_role, session
from sagemaker.s3 import S3Downloader, S3Uploader, s3_path_join
from pathlib import Path

account_id = boto3.client("sts").get_caller_identity().get("Account")
sm_session = session.Session()
region = sm_session._region_name
role = get_execution_role()
bucket = sm_session.default_bucket()

current_dir = os.getcwd()

prefix = "sagemaker/abalone/models/byoc"
default_bucket_prefix = sm_session.default_bucket_prefix

# If a default bucket prefix is specified, append it to the s3 path
if default_bucket_prefix:
    prefix = f"{default_bucket_prefix}/{prefix}"

predictor_image_name = "abalone/predictor"

pretrained_xgboost_model_s3uri = (
    f"s3://sagemaker-example-files-prod-{region}/models/xgb-abalone/xgboost-model"
)

base_dir = Path("../data").resolve()
predictor_model_dir = Path("./models").absolute()

if not predictor_model_dir.exists():
    predictor_model_dir.mkdir()

In [ ]:
# Download pretrained xgboost model
!aws s3 cp $pretrained_xgboost_model_s3uri $predictor_model_dir

In [ ]:
!pygmentize ./code/inference.py

### Build and test custom inference image locally

 - [Dockerfile](./Dockerfile) implementation
   - Set required LABEL `LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true`
   - Installs required software and python packages
   - Copies all files under code directory to `/opt/program`
   - Sets `ENTRYPOINT` to `["python"]`
   - Sets `CMD` to `["serve"]` (python script that launches nginx, gunicorn in the background)

In [ ]:
!pygmentize Dockerfile

In [ ]:
# build image locally
!docker build -t $predictor_image_name .

### Launch and test custom Inference container locally

In [ ]:
# open a terminal and cd into the predictor directory (the location where predictor.ipynb is located) run the following command
# run this command to launch container locally
# mounts the [models](./models) directory to `/opt/ml/model` directory inside the container and maps container port `8080` to host port `8080`
# docker run --rm -v $(pwd)/models:/opt/ml/model -p 8080:8080 abalone/predictor

#### Check container health by invoking `/ping`

If the `/ping` was successful you should see a response similar to `"GET /ping HTTP/1.1" 200 1` in the terminal

In [ ]:
# Ping local inference endpoint
# !curl http://localhost:8080/ping

### Verify container logs locally (using docker logs)

- To inspect a running container to view container config values or IP address we use `docker inspect <CONTAINER_ID_OR_NAME>`
- To view and tail logs generated in the container we use `docker logs --follow <NUM_OF_LINES> <CONTAINER_ID_OR_NAME>`
- SageMaker publishes container logs to CloudWatch. CloudWatch logs for a given endpoint are published to the following log stream path
`/aws/sagemaker/Endpoints/ENDPOINT_NAME/VARIANT_NAME/CONTAINER_NAME`

**NOTE:** 
1. Run this command in a terminal as running this inside a cell would hang execution.
1. the below command assumes there is only one running container. If you have more, then use command with container name `docker inspect <CONTAINER_ID_OR_NAME>` 

In [ ]:
# RUN THE BELOW IN A SEPARATE NEW TERMINAL
# docker ps --format "{{.Names}}" | xargs -n1 -I{} docker logs --follow --tail 50 {}

### Troubleshooting container locally (using logs)

`!docker logs abalone/featurizer`

#### Test records for inference

Grab a test record from [abalone_test_predictor.csv](../featurizer/data/abalone_test_predictor.csv), generated by [`featurizer.ipynb`](../featurizer/featurizer.ipynb), format it as a CSV record, and send it as raw data to the endpoint `http://localhost:8080/invocations` path

Uncomment below cells to test inference on local container.

In [ ]:
# Send test records to /invocations on the endpoint
# !curl --data-raw '-1.3317586042173168,-1.1425409076053987,-1.0579488602777858,-1.177706547272754,-1.130662184748842,-1.1493955859050584,-1.139968767909096,0.0,1.0,0.0' \
# -H 'Content-Type: text/csv; charset=utf-8' \
# -v http://localhost:8080/invocations

In [ ]:
# Send test records to /invocations on the endpoint
# !curl --data-raw '0.7995425613971686,0.877965470587042,1.326659055767273,1.398563012556441,0.9896192483949702,1.509166873607132,2.01650402614155,0.0,0.0,1.0' \
# -H 'Content-Type: text/csv; charset=utf-8' \
# -v http://localhost:8080/invocations

### Tag and push the local image to private ECR

- Tag the `abalone/predictor` local image to `{account_id}.dkr.ecr.{region}.amazonaws.com/{imagename}:{tag}` format
- Run [./build_n_push.sh](./build_n_push.sh) shell script with image name `nginx` as parameter


In [ ]:
!chmod +x ./build_n_push.sh

!./build_n_push.sh abalone/predictor

### Optional: Test predictor inference image by deploying to a real-time endpoint

- **Step 1:** Compress your model in `./models/xgboost-model` to `model.tar.gz` format and upload to s3
- **Step 2:** Create Model object with your custom inference image and deploy model to endpoint
- **Step 3:** Send test inference request to deployed endpoint
- **Step 4:** Cleanup

In [ ]:
import subprocess

os.chdir(predictor_model_dir)

model_s3uri = s3_path_join(f"s3://{bucket}/{prefix}", "predictor")

predictor_model_path = predictor_model_dir.joinpath("model.tar.gz")

if predictor_model_path.exists():
    predictor_model_path.unlink()

# SageMaker expects model artifacts to be compressed to `model.tar.gz`
tar_cmd = "tar -czvf model.tar.gz xgboost-model ../code/"
result = subprocess.run(tar_cmd, shell=True, capture_output=True)

if result.returncode == 0:
    print(f"{predictor_model_path} archive created successfully!")
    os.chdir(current_dir)
else:
    os.chdir(predictor_model_dir)
    print("An error occurred:", result.stderr)

In [ ]:
# Upload compressed model artifact to S3 using S3Uploader utility class
model_data_url = S3Uploader.upload(
    local_path=predictor_model_path.absolute(),
    desired_s3_uri=model_s3uri,
    sagemaker_session=sm_session,
)
print(f"Uploaded predictor model.tar.gz to {model_data_url}")

#### **Step 2:** Create and deploy model with predictor inference image

In [ ]:
from datetime import datetime
from uuid import uuid4
from sagemaker.model import Model

ecr_image = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{predictor_image_name}:latest"

suffix = f"{str(uuid4())[:5]}-{datetime.now().strftime('%d%b%Y')}"

model_name = f"AbaloneXGB-predictor-{suffix}"
predictor_model = Model(
    image_uri=ecr_image,
    name=model_name,
    model_data=model_data_url,
    role=role,
    sagemaker_session=sm_session,
)

endpoint_name = f"Abalone-nginx-ep-{suffix}"
print(f"Deploying model {model_name} to endpoint: {endpoint_name}")
predictor = predictor_model.deploy(
    endpoint_name=endpoint_name, initial_instance_count=1, instance_type="ml.m5.xlarge"
)

### **Step 3:** Send test inference requests to deployed endpoint

We use sample transformed records from featurizer [abalone_featurizer_predictions.csv](./abalone_featurizer_predictions.csv) for predictions.

In [ ]:
from time import sleep
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

test_records = Path("./abalone_featurizer_predictions.csv")

predictor = Predictor(endpoint_name=endpoint_name, sagemaker_session=sm_session)
predictor.content_type = "text/csv; charset=utf-8"
predictor.serializer = CSVSerializer()
predictor.accept = "application/json"
predictor.deserializer = JSONDeserializer()

# Send 50 records for inference
limit = 10
i = 0

with open(test_records, "r") as _f:
    lines = _f.readlines()
    for row in lines:
        if i <= limit:
            prediction = None
            print(f"input: {row}")
            try:
                response = predictor.predict(row)
                print(response)
                i += 1
                sleep(0.15)
            except Exception as e:
                print(f"Prediction error: {e}")
                pass

### View logs emitted by the endpoint in CloudWatch

In [ ]:
from datetime import timedelta

logs_client = boto3.client("logs")
end_time = datetime.utcnow()
start_time = end_time - timedelta(minutes=15)

log_group_name = f"/aws/sagemaker/Endpoints/{endpoint_name}"
log_streams = logs_client.describe_log_streams(logGroupName=log_group_name)
log_stream_name = log_streams["logStreams"][0]["logStreamName"]

# Retrieve the logs
logs = logs_client.get_log_events(
    logGroupName=log_group_name,
    logStreamName=log_stream_name,
    startTime=int(start_time.timestamp() * 1000),
    endTime=int(end_time.timestamp() * 1000),
)

# Print the logs
for event in logs["events"]:
    print(f"{datetime.fromtimestamp(event['timestamp'] // 1000)}: {event['message']}")

### Cleanup

Cleanup resources. Delete endpoint and model

In [ ]:
# Delete model
try:
    print(f"Deleting model: {model_name}")
    predictor.delete_model()
except Exception as e:
    print(f"Error deleting Model: {model_name}\n{e}")
    pass

# Delete endpoint
try:
    print(f"Deleting endpoint: {endpoint_name}")
    predictor.delete_endpoint()
except Exception as e:
    print(f"Error deleting EP: {endpoint_name}\n{e}")
    pass

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/realtime|byoc|byoc-nginx-python|featurizer|predictor.ipynb)
